In [ ]:
#### load packages, *.h5 folder and exp.h5 file location
%matplotlib widget

from essential_func import *
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

samples_csv                = 'data_directory.csv'
default_sample_dir         = 'July-2021-Sample#6'      # 'July-2021-Sample#6'    '21-Nov'   'July-sorted' 
csv_and_code_abs_directory = '/Users/bashit.a/Documents/Alzheimer/Codes/'

dropdown_name_list, cwd, exp_folder = change_python_path(dropdown_name = default_sample_dir, csv_file_location = csv_and_code_abs_directory, samples_csv = samples_csv)

In [5]:
### show heat map for any file -- (Independent cell) - files are required to have azimuthal averaged
%matplotlib widget
from ipywidgets import fixed

default_saxs_inc = 0.001
default_waxs_inc = 0.001

dropdown_dir   = ipywidgets.Dropdown(options= dropdown_name_list,           value = default_sample_dir,              description='Folder ',       disabled=False)
dropdown_file  = ipywidgets.Dropdown(options= cwd_files_search_with('.h5'), value = cwd_files_search_with('.h5')[0], description='File ',         disabled=False)
cmap           = ipywidgets.Dropdown(options= cmap_list(),                  value = 'jet',                           description='Color Map ',    disabled=False)
rep_value      = ipywidgets.Dropdown(options= ['circ', 'point-WAXS', 'point-SAXS'], value = 'circ',                  description='Rep ',          disabled=False)
data_binning   = ipywidgets.Checkbox(value  = True,                 style= {'description_width': 'initial'}   ,      description='Data Binning ', disabled=False)
q_value_widget = ipywidgets.FloatSlider(value = 1.34, min= qgrid2[0], max=qgrid2[-1], step=0.001,       description='q Value:', disabled=False, continuous_update=False, layout=ipywidgets.Layout(width='25%'), readout=True, readout_format='.4f', style= {'description_width': 'initial'})    
q_value_widget.layout.visibility = 'hidden'                                 # 'hidden' or 'visible'

saxs_max_widget = ipywidgets.FloatSlider(value = 10,     min= 0,       max= 500, step=1 ,                   description='SAXS Max:',  continuous_update=False, layout=ipywidgets.Layout(width='25%'), readout=True, readout_format='.4f', style= {'description_width': 'initial'})    
saxs_inc_widget = ipywidgets.FloatSlider(value = default_saxs_inc,     min= 0.0001, max= 200, step=0.001 ,  description='SAXS Inc.:', continuous_update=False, layout=ipywidgets.Layout(width='25%'), readout=True, readout_format='.4f', style= {'description_width': 'initial'})    
waxs_max_widget = ipywidgets.FloatSlider(value = 0.035,  min= 0.001,   max= 1, step=0.001 ,                 description='WAXS Max:',  continuous_update=False, layout=ipywidgets.Layout(width='25%'), readout=True, readout_format='.4f', style= {'description_width': 'initial'})    
waxs_inc_widget = ipywidgets.FloatSlider(value = default_waxs_inc,     min= 0.0001, max= 0.035, step=0.001, description='WAXS Inc.:', continuous_update=False, layout=ipywidgets.Layout(width='25%'), readout=True, readout_format='.4f', style= {'description_width': 'initial'})    

### set saxs-waxs max, value, step update according to file
def set_saxs_waxs_widget(diff_patterns, scattering):

    if scattering == '_SAXS':
        ### entend widget max just to avoid error when setting min        
        if np.min(diff_patterns) > saxs_max_widget.max:
            saxs_max_widget.max   = np.max(diff_patterns)
        
        saxs_max_widget.min   = np.min(diff_patterns)
        saxs_max_widget.max   = np.max(diff_patterns)
        saxs_max_widget.step  = saxs_max_widget.max/100

        saxs_inc_widget.max   = saxs_max_widget.value
        saxs_inc_widget.step  = saxs_max_widget.value/1000

    elif scattering == '_WAXS2':
        ### entend widget max just to avoid error when setting min
        if np.min(diff_patterns) > waxs_max_widget.max:
            waxs_max_widget.max   = np.max(diff_patterns)
            
        waxs_max_widget.min   = np.min(diff_patterns)
        waxs_max_widget.max   = np.max(diff_patterns)
        waxs_max_widget.step  = waxs_max_widget.max/100

        waxs_inc_widget.max   = waxs_max_widget.value
        waxs_inc_widget.step  = waxs_inc_widget.max/1000

def saxs_waxs_max_inc_visibility(saxs_max, saxs_inc, waxs_max, waxs_inc):
    saxs_max_widget.layout.visibility = saxs_max
    saxs_inc_widget.layout.visibility = saxs_inc
    waxs_max_widget.layout.visibility = waxs_max
    waxs_inc_widget.layout.visibility = waxs_max

def heat_map(file, cmap, data_binning, saxs_max, saxs_inc, waxs_max, waxs_inc, rep_value, q_value):

    if rep_value == 'circ':
        # saxs wax max-inc visibility
        saxs_waxs_max_inc_visibility(saxs_max = 'visible', saxs_inc = 'visible', waxs_max = 'visible', waxs_inc = 'visible')
        f = plt.figure(figsize=(17,6), constrained_layout=False)
        ax = f.subplots(nrows=1, ncols=2, )
        q_value_widget.layout.visibility = 'hidden' # 'hidden' or 'visible'
        
        for scattering in ('_SAXS','_WAXS2'):
            diff_patterns = find_rep_value(qgrid2, read_Iq(file, scattering) , method = 'circ')
            set_saxs_waxs_widget(diff_patterns, scattering)
        
        f = plot_heat_map_from_file(file, qgrid2, scatterings = ('_SAXS','_WAXS2'), heatmap_rep_value = 'circ', arg_qvalue = None, cmap = cmap, args = (f,ax), data_binning=data_binning, bins = [np.fromiter(drange(0, saxs_max, saxs_inc), float), np.fromiter(drange(0, waxs_max, waxs_inc), float)] )  # discrete_cmap(N=5, base_cmap = 'cubehelix')     viridis

    elif rep_value.find('point') == 0:              # x = point-waxs or x = point-saxs then x.find('point') =0
        f = plt.figure(figsize=(17,6), constrained_layout=False)
        ax = f.subplots(nrows=1, ncols=2, )
        q_value_widget.layout.visibility = 'visible' # 'hidden' or 'visible'
        scattering = '_WAXS2' if rep_value == 'point-WAXS' else '_SAXS'

        # q value min-max defining by choosing first frame
        Iq = read_Iq(file, scattering, frame = 0)
        idx_l, idx_u, _ = valid_idx_search(qgrid2, Iq.reshape(1,-1))
        print(f'{scattering} Range q ~ {qgrid2[idx_l]:0.4f} to {qgrid2[idx_u-1]:0.4f}')
        q_value_widget.min   = qgrid2[idx_l]
        q_value_widget.max   = qgrid2[idx_u-1]
        q_value_widget.value = np.percentile(qgrid2[idx_l:idx_u],15) if q_value < q_value_widget.min or q_value > q_value_widget.max else q_value
        print(f'q widget value is : {q_value_widget.value:0.4f}, function qvalue is : {q_value:0.4f}')
        
        Iq = read_Iq(file, scattering)
        diff_patterns = find_rep_value(qgrid2, Iq , args=q_value_widget.value, method = 'point')
        set_saxs_waxs_widget(diff_patterns, scattering)

        # saxs wax max-inc visibility
        saxs_waxs_max_inc_visibility(saxs_max = 'hidden', saxs_inc = 'hidden', waxs_max = 'visible', waxs_inc = 'visible') if scattering == '_WAXS2' else saxs_waxs_max_inc_visibility(saxs_max = 'visible', saxs_inc = 'visible', waxs_max = 'hidden', waxs_inc = 'hidden')
        bins = np.fromiter(drange(0, waxs_max, waxs_inc), float) if scattering == '_WAXS2' else np.fromiter(drange(0, saxs_max, saxs_inc), float)
        f = plot_heat_map_from_file(file, qgrid2, scatterings = (scattering,), heatmap_rep_value = 'point', arg_qvalue = q_value, cmap = cmap, args = (f,ax[0]), data_binning=data_binning, bins = bins )  # discrete_cmap(N=5, base_cmap = 'cubehelix')     viridis

        q_values = qgrid2[idx_l:idx_u-1]
        stds = np.zeros_like(q_values)
        for q_ in range(len(q_values)):
            stds[q_] = np.std(Iq[:,q_])
        
        ax[1].clear()
        ax[1].plot(q_values, stds,'blue')
        idx = qgrid_to_indices(q_values, qvalue=q_value)
        ax[1].scatter(q_values[idx], stds[idx],color='red', marker = 'o')
        ax[1].set(xlabel = "q", ylabel = "std")
        
    print(f'File = {file}, SAXS-MAX = {saxs_max:0.3f}, SAXS-INC = {saxs_inc:0.3f}, WAXS-MAX = {waxs_max:0.3f}, WAXS-INC = {waxs_inc:0.3f}, cmap = {cmap}')
    print(f'File = {file}, Width, Height = {width_height(file)}')
    plt.tight_layout()

### change dropdown_file lists when directory is changed
def update_samples_list(*args):
    global cwd, exp_folder
    _, cwd, exp_folder = change_python_path(dropdown_name = dropdown_dir.value, csv_file_location = csv_and_code_abs_directory, samples_csv = samples_csv)
    dropdown_file.options = cwd_files_search_with('.h5')
    dropdown_file.value = cwd_files_search_with('.h5')[0]
dropdown_dir.observe(update_samples_list)

ui  = ipywidgets.VBox([ipywidgets.HBox([dropdown_dir, dropdown_file, cmap, data_binning, rep_value, q_value_widget]), ipywidgets.HBox([saxs_max_widget, saxs_inc_widget, waxs_max_widget, waxs_inc_widget])])
out = ipywidgets.interactive_output(heat_map, {'file' : dropdown_file, 'cmap' : cmap, 'data_binning' : data_binning, 'saxs_max' : saxs_max_widget, 'saxs_inc' : saxs_inc_widget , 'waxs_max' : waxs_max_widget, 'waxs_inc' : waxs_inc_widget, 'rep_value' : rep_value, 'q_value' : q_value_widget})
# out = ipywidgets.interact(heat_map, file = dropdown_file, cmap=cmap, data_binning=data_binning, saxs_max=saxs_max_widget, saxs_inc=saxs_inc_widget , waxs_max=waxs_max_widget, waxs_inc=waxs_inc_widget, rep_value=rep_value, q_value=q_value_widget)
display(out, ui)

Output()

In [26]:
### Heatmap
%matplotlib widget
from scipy import signal
file = '2428_FC-roi1.h5' #'2029_Dentate_gyrus-roi0_masked.h5'
window_size = 4
scattering = '_WAXS2'
directory = '/Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-1/' # '/Users/bashit.a/Documents/Alzheimer/Dec-2020/'#'/Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-6/'
Iq = read_Iq(file, scattering, directory=directory)      # Iq shape (3721, 690)
Iq = Iq[:,310:334]
Width, Height = width_height(file ,directory=directory)
window  = np.ones(window_size)/window_size

Iq_f = np.zeros_like(Iq);
for frame in range(len(Iq)):
    Iq_f[frame] = np.convolve(window, Iq[frame], 'same')                   # outputs same length output and will be used for area and plotting
Iq_new = Iq - Iq_f

diff_patterns = find_rep_value(qgrid2, Iq_f, method = 'circ')  
img_orig = snaking(Width, Height, diff_patterns)

f, ax = plt.subplots(1, 1, num=f'{file} Heat maps', figsize=(10,5))
plot_heat_map_from_data(img_orig, Width, Height, args=(f,ax), title= None, cmap=discrete_cmap(N=10, base_cmap = 'cubehelix'))

Minimum, Maximum Iq : 0.0264, 13658896.0837


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
### load pixalated sum either directly from h5 file or already stored pixalted sum npz file -------- ##### (Independent Cell) - loading from file

# spec
file = '2428_FC-roi1.h5'
load_from = 'npz' # 'h5'

### semi spec
folder = 'pixalated_sum_waxs'
if load_from == 'h5':
    ##### Load file before WAXS thresholding (Requires to run the folliwing cell only) - a 300x300 roi should not take more than 3 min to load for M1 macbook 
    dset_waxs_sum = pixalated_sum_waxs(file, save_as_file=False)

elif load_from == 'npz':
    ### load pixalated sum file (2048_B8.h5-pixalated_sum_waxs.npz) from the saved folder (pixalated_sum_waxs)
    try:
        arr = np.load(f'{folder}/{file.strip(".h5")}-pixalated_sum_waxs.npz', allow_pickle=True)
        dset_waxs_sum      = arr['waxs_sum']
        dset_waxs_sum_stat = arr['waxs_sum_stat']
        dset_waxs_sum_df   = arr['waxs_sum_percentile']
        print(dset_waxs_sum_stat)
        print(dset_waxs_sum_df)
    except:
        print(f'{folder}/{file}-pixalated_sum_waxs.npz file not found')
else:
    print('Something Went Wrong')
    

['Pixalated Sum WAXS shape = ' (1043, 981) 'Min. = -7813.000000'
 'Mean = 17521.151828' 'Median = 10798.000000' 'Max. = 232937462.000000']
[[70 '12,649.00']
 [71 '12,727.00']
 [72 '12,803.00']
 [73 '12,878.00']
 [74 '12,953.00']
 [75 '13,028.00']
 [76 '13,103.00']
 [77 '13,178.00']
 [78 '13,255.00']
 [79 '13,332.00']
 [80 '13,410.00']
 [81 '13,497.00']
 [82 '13,589.00']
 [83 '13,692.00']
 [84 '13,809.00']
 [85 '13,939.00']
 [86 '14,076.52']
 [87 '14,222.00']
 [88 '14,368.00']
 [89 '14,509.00']
 [90 '14,644.80']
 [91 '14,776.00']
 [92 '14,902.00']
 [93 '15,030.00']
 [94 '15,162.00']
 [95 '15,318.00']
 [96 '15,539.00']
 [97 '16,098.00']
 [98 '17,745.00']
 [99 '23,414.00']]


In [28]:
### USE this cell if and only iff any frame is causing high value to rise in pixalated sum, ex. 2428_FC-roi1.h5, frame 3740 gives rise to high values
%matplotlib widget
frame = 3740
with h5py.File(file,'r') as hdf:      
    dset = hdf.get(f'{h5_top_group(file)}/primary/data')
    dset_waxs = dset['pilW2_image'][frame]
plt.imshow(dset_waxs_sum-dset_waxs)
dset_waxs_sum = dset_waxs_sum-dset_waxs

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
## WAXS thresholding and loading ------------ (Dependent Cell - requires running either of the previou cells deping on loading pixalted sum on RAM or loading from file)
## you don't need azimuthal averaging of the original file to do masking
## specs/Slider Properties
AMIN_DEFAULT_VALUE = 0       # minimum value for the pixalated sum
AMAX_DEFAULT_VALUE = np.median(dset_waxs_sum) ; AMAX_MIN_VALUE = dset_waxs_sum.min() ; AMAX_MAX_VALUE = np.percentile(dset_waxs_sum, 99);  AMAX_SLIDER_STEP = 100;    # maximum values for the pixalated sum
# valid_range_min, valid_range_max = (0,10)   # just for WAXS Display purpose

n_patterns = list(map(lambda x,y: x*y, *zip(width_height(file))))[0]   # calculate number of frames


def waxs_thresholding(a_min, a_max, thr, frame):
    %matplotlib widget
    
    # load waxs frame data 
    with h5py.File(file,'r') as hdf:
        dset_waxs = hdf.get(f'{h5_top_group(file)}/primary/data/pilW2_image')         # waxs data read from h5 file
        dset_waxs = np.array(dset_waxs[frame])
        
    args = (dset_waxs_sum, a_min, a_max, thr)
    dset_waxs_thr, gray_img, thr_fr_img = threshold_patch_one_frame(dset_waxs, args = args)
    
    #### plot thresholded pixalated sum
    f, axs = plt.subplots(nrows=2,ncols=2,figsize=(12,8),num=f'{file}')
    im = axs[0,0].imshow(dset_waxs_thr, cmap = 'rainbow')       #plt.colorbar(im, fraction=0.046, pad=0.04)  
    show_colorbar(im,f,axs[0,0])
    axs[0,0].set_title('Pixelated Sum')
    #### plot global thresholding  
    im = axs[0,1].imshow(gray_img, cmap='gray')       
    axs[0,1].set_title(f'Binary Threshold = {thr}')

    patching(file, frame, qgrid2, args=args, axes = (f,[axs[1,0], axs[1,1]]), method = 'thresholding', exp_folder = exp_folder  )

    print('a_min = ', a_min, 'a_max = {:0.2f}'.format(a_max), 'thr = ', thr)

frame = ipywidgets.IntSlider(value=0, min=0, max=n_patterns-1, step=1, continuous_update=False, layout=ipywidgets.Layout(width='50%'))
a_min = ipywidgets.fixed(value=AMIN_DEFAULT_VALUE)
a_max = ipywidgets.FloatSlider(value=AMAX_DEFAULT_VALUE, min=AMAX_MIN_VALUE,     max=AMAX_MAX_VALUE, step=AMAX_SLIDER_STEP, continuous_update=False, layout=ipywidgets.Layout(width='50%'))
thr   = ipywidgets.IntSlider(value=a_min.value,          min=AMIN_DEFAULT_VALUE, max=AMAX_MAX_VALUE, step=AMAX_SLIDER_STEP, continuous_update=False, layout=ipywidgets.Layout(width='50%'))
ipywidgets.jslink((a_max,'value'),(thr,'max'))           # make sure threshold's maximum value doesn't exceed a_max value

ipywidgets.interact(waxs_thresholding, a_min=a_min, a_max=a_max, thr=thr, frame=frame)

interactive(children=(FloatSlider(value=10798.0, continuous_update=False, description='a_max', layout=Layout(w…

<function __main__.waxs_thresholding(a_min, a_max, thr, frame)>

In [78]:
##### --------- 1-d averaging after thresholding for one file  --------- (Independent Cell)
### spec
source_file = '2346_Dentate_gyrus-roi1.h5'
args =  0, 19888, 17000   # manually can set args = (a_min.value, a_max.value, thr.value) = 0, 30000, 16000 or Get values from 3rd cell print output

### computation
masked_file = circ_avg_from_patches(source_file, qgrid2, args=args, method = 'thresholding', exp_folder = exp_folder  )

2346_Dentate_gyrus-roi1_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-6 ...
2346_Dentate_gyrus-roi1_masked.h5 copy is done
2346_Dentate_gyrus-roi1_masked.h5 Loading data into a numpy array started 
2346_Dentate_gyrus-roi1_masked.h5 Loading data into a numpy array finished in 44.771737875000326 seconds
2346_Dentate_gyrus-roi1_masked.h5 Patching Started 
2346_Dentate_gyrus-roi1_masked.h5 Patching finished in 45.28245754100135 seconds
2346_Dentate_gyrus-roi1_masked.h5 patched pilW2_image dataset creation staring ... 
2346_Dentate_gyrus-roi1_masked.h5 patched pilW2_image dataset creation finished in 99.11726783300037 seconds
Circular averaging starts now ... 
data received: sn=2346_Dentate_gyrus-roi1, fr1=0
data received: sn=2346_Dentate_gyrus-roi1, fr1=488
data received: sn=2346_Dentate_gyrus-roi1, fr1=976
data received: sn=2346_Dentate_gyrus-roi1, fr1=1464
data received: sn=2346_Dentate_gyrus-roi1, fr1=1952
data received: sn=2346_Dentate_gyru

OSError: Can't write data (file write failed: time = Fri Aug 13 22:14:50 2021
, filename = '2346_Dentate_gyrus-roi1_masked.h5', file descriptor = 73, errno = 28, error message = 'No space left on device', buf = 0x3697f0000, total write size = 43122240, bytes this sub-write = 43122240, bytes actually written = 18446744073709551615, offset = 0)

In [35]:
############### -------- configure patch size and copy the pathces tuplte to the next cell for computing 1-d averaging for all frames ------------- ################### (Independent Cell)
%matplotlib widget
frame = 1326         # possible_bkg frame
file = '2048_B8_masked.h5'
patches = patches = [
#                              [[800, 0], [960, 120]],
#                              [[0, 0], [960, 120]],
#                              [[590, 220], [610, 240]],
#                              [[0, 590], [45, 615]],
#                              [[315, 740], [345, 765]],
#                              [[630, 900], [655, 920]],
#                              [[20, 955], [70, 985]],
#                              [[890, 375], [920, 400]],
#                              [[865, 990], [985, 1030]],
#                              [[310, 420], [330, 435]]
                                [[380,970],[430,1030]]
                    ]

patching(file, frame, qgrid2, args=patches, method = 'rec_circ_patch');

frame information extraction completes with _SAXS shape (1, 1043, 981) _WAXS shape (1, 1043, 981)...
Circular averaging starts now ... 
2048_B8_masked_1326.h5 total 1-d averaging time 0.45188058298663236 seconds


/Users/bashit.a/Documents/Alzheimer/Codes/py4xs/data2d.py:350: RuntimeWarning: invalid value encountered in sqrt
  dI = np.sqrt(Iq2-Iq*Iq)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# 1-d averaging after rectangular and/or circular patches for one file  (Independent Cell)
### spec
source_file = "2048_B8.h5"
patches = [
#                              [[800, 0], [980, 150]],
#                              [[245, 0], [385, 130]],
          ]

### computation
masked_file = circ_avg_from_patches(source_file, qgrid2, patches)

DID not PATCH 2017_EC-roi3_masked.h5 already exists - did not copy, 
1-D Averaging is NOT performed as 2017_EC-roi3_masked.h5 already exist 
TO do patching on 2017_EC-roi3_masked.h5 delete the file first manually and run this cell again


In [2]:
## patching information retrival from _masked.h5 file    (Independent Cell)
#specs
file = '2346_EC-roi0_masked.h5'
method = 'thresholding'

# computation
try: patches = get_patch_attributes(f'{file}', method = 'thresholding') if method == 'thresholding' else get_patch_attributes(f'{file}'); print(patches)
except: print(f'{file} does not have patches information on the processed folder')

2346_EC-roi0_masked.h5 does not have patches information on the processed folder


In [20]:
### check config.json file integrity     (Independent Cell) - no error no problem
No_error_no_prb = get_json_str_data('sample-6-july-2021-withthreshold.json');

In [3]:
# 1-d averaging after patches for ALL files in the current directory   (Independent Cell)

# specs
data = get_json_str_data("config-mar-24-withthreshold.json")
method = 'thresholding'                                        # 'thresholding' or 'rec_circ_patch'

### do masking and 1-d averaging (Lin Yang's code by BNL used here for 1-d averaging)
files = []

for file in data['files']:
    source_file = file['name']
    try: 
        if method == 'rec_circ_patch':
            patches = file['patches']
            masked_file = circ_avg_from_patches(source_file, qgrid2, patches) 
        elif method == 'thresholding': 
            patches = file['threshold']
            print(file, 'thresholded to -->', patches)
            masked_file = circ_avg_from_patches(source_file, qgrid2, args=tuple(patches), method = 'thresholding')  # args must be of tuple
    except: continue
    files.append((source_file, masked_file))

print(f'Successfully created (source file, masked file) \n\n')
[print(idx+1, '. ', s, '--->' ,t) for idx, (s,t) in enumerate(files)]
pass

{'name': '1886_EC-roi0.h5', 'possible_plaque': None, 'possible_bkg': 424, 'patches': [], 'threshold': [0, 44393, 15210], 'comments': ''} thresholded to --> [0, 44393, 15210]
1886_EC-roi0_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/Mar-24/BNL-Data ...
1886_EC-roi0_masked.h5 copy is done
1886_EC-roi0_masked.h5 Loading data into a numpy array started 
1886_EC-roi0_masked.h5 Loading data into a numpy array finished in 55.423180167 seconds
1886_EC-roi0_masked.h5 Patching Started 
1886_EC-roi0_masked.h5 Patching finished in 46.708706666 seconds
1886_EC-roi0_masked.h5 patched pilW2_image dataset creation staring ... 
1886_EC-roi0_masked.h5 patched pilW2_image dataset creation finished in 89.06730066699998 seconds
Circular averaging starts now ... 
data received: sn=1886_EC-roi0, fr1=0
data received: sn=1886_EC-roi0, fr1=480
data received: sn=1886_EC-roi0, fr1=960
data received: sn=1886_EC-roi0, fr1=1440
data received: sn=1886_EC-roi0, fr1=1920
data received: sn=1886_EC-r

In [29]:
### 1d circularly averaged data for masked file
%matplotlib widget
import ipywidgets   # from IPython.display import display; display(amp)
import hdf5plugin

# specs 
source_file = '2428_FC-roi1_masked.h5'          # must input source file name
# masked_file = "2109_EC-roi1.h5"          # comment out to avoid plotting
scattering = ('_SAXS', '_WAXS2',)

# computation
# anyinitialized masked_file will be removed beginning of the cell
try : del masked_file
except NameError: pass

Width, Height = width_height(source_file)    # considering the fact that width and height is same for source and masked file
n_patterns = Width*Height

f,axs = plt.subplots(nrows=2, ncols=3, figsize=(16,8))
# plot_heat_map_from_file(source_file, qgrid2, scatterings = (scattering[1],), cmap = 'viridis', args = (f, axs[0,1]))
plot_heat_map_from_file(source_file, qgrid2, scatterings = (scattering[1],) , cmap = 'jet', args = (f, axs[0,2]), data_binning=False, bins = np.fromiter(drange(0, 0.04, 0.001), float) )  # discrete_cmap(N=5, base_cmap = 'cubehelix')     viridis
plot_heat_map_from_file(source_file, qgrid2, scatterings = (scattering[0],) , cmap = 'jet', args = (f, axs[1,2]), data_binning=False, bins = np.fromiter(drange(0, 20, 0.5), float) )  # discrete_cmap(N=5, base_cmap = 'cubehelix')     viridis

def update_plot(frame):
    axs[0,0].clear()
    axs[1,0].clear()
    
    axs[0,0].set_yscale('log')
    axs[1,0].set_yscale('log')
    
    axs[0,0].set_xlabel('q'); axs[0,0].set_ylabel('log(I)');
    axs[1,0].set_xlabel('q'); axs[1,0].set_ylabel('log(I)');
    
    Iq_S_SAXS = read_Iq(source_file, scattering[0])   # Source must be read even you do mask or not
    Iq_S_WAXS = read_Iq(source_file, scattering[1])   # Source must be read even you do mask or not

    try: 
        masked_file
        if source_file ==  masked_file:
            axs[0,0].plot(qgrid2, Iq_S_WAXS[frame], label = f' Fr- {frame}');
            waxs_diff_image(source_file, frame, f, axs[0,1])

            axs[1,0].plot(qgrid2, Iq_S_SAXS[frame], label = f' Fr- {frame}');
            saxs_diff_image(source_file, frame, f, axs[1,1])

        else:
            Iq_M = read_Iq(masked_file, scattering)
            axs[0,0].plot(qgrid2, Iq_S[frame], label = f'Before Mask Fr- {frame}'); 
            axs[0,0].plot(qgrid2, Iq_M[frame], label = f'After Mask Fr- {frame}');
            waxs_diff_image(file, frame, f, axs[0,1])


    except NameError: 
        axs[0,0].plot(qgrid2, Iq_S_WAXS[frame], label = f' Fr- {frame}');
        waxs_diff_image(source_file, frame, f, axs[0,1])

        axs[1,0].plot(qgrid2, Iq_S_SAXS[frame], label = f' Fr- {frame}');
        saxs_diff_image(source_file, frame, f, axs[1,1])

    else: print('Something Wrong!')

    axs[0,0].legend()
    axs[1,0].legend()
    plt.suptitle(f'{source_file}')
    plt.tight_layout()

frame = ipywidgets.IntSlider(min=0, max=n_patterns-1, value=0, description = "Frame")
ipywidgets.interactive(update_plot, frame=frame)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:200: RuntimeWarning: Mean of empty slice
  diff_patterns[frame] = np.nanmean(valid_diff_values[frame])  # calculate the mean of array ignoring the NaN value


Minimum, Maximum Iq : nan, nan
Minimum, Maximum Iq : 1.4404, 232.5941


interactive(children=(IntSlider(value=0, description='Frame', max=3905), Output()), _dom_classes=('widget-inte…

In [8]:
file = "2029_dentate_gyrus_mica-roi0.h5"
import h5py
import numpy as np

# dset = h5py.File(file,'r')

# dset = dset['2029_Dentate_gyrus-roi0/primary/data']  # VSTOXX futures data
# dset_saxs = dset['pil1M_image']
# dset_saxs = np.array(dset_saxs)
# dset.close()

with h5py.File(file,'r') as hdf:
    dset = hdf.get('2029_dentate_gyrus_mica-roi0/primary/data')
    dset_saxs = dset['pil1M_image']
    dset_saxs = np.array(dset_saxs)


OSError: Can't read data (can't open directory: /Users/bashit.a/miniforge3/envs/py38/lib/hdf5/plugin)

In [3]:
################# --------- Find Q range to plot heat maps --------- #################
%matplotlib widget
import ipywidgets         # from IPython.display import display; display(amp)
from ipywidgets import fixed

# semi-specs
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

### q_low, q_high, q_low - idx[2] and q_high + idx[2] for poly fit, number of points
indices = ((0.115, 0.19, 13,   48, '_SAXS' , 'bumpy'),      # 72, 81
           (1.48,  1.57, 10, 4.15, '_WAXS2', 'paraffin'),   # 345, 355
           (1.30,  1.42, 10,  4.7, '_WAXS2', 'amyloid'),    # 309, 323
           (1.89,  1.93,  7,  3.2, '_WAXS2', 'mica')        # 427, 433
          )

## polyhit with heatmap from drop down list
files = cwd_files_search_with('.h5')
dropdown = ipywidgets.Dropdown(options=files, value = files[0],description='File : ', disabled=False)
ipywidgets.interactive(file_polyfit_heatmap_plot, file = dropdown, indices = fixed(indices), qgrid2 = fixed(qgrid2))

## polyhit with heatmap for one file
# file = '2048_B8.h5'
# file_polyfit_heatmap_plot(file, indices, qgrid2)

interactive(children=(Dropdown(description='File : ', options=('1892_High_Molecular_Tau-roi0.h5', '1892_High_M…

In [2]:
### Merge all PDFs in the directory specified
### you can put all your pdfs that you want to merge to the PDF folder
from essential_func import *
pdfs_merging(directory = '/PDF', output = 'result.pdf')

In [33]:
### Show merged data on multiple files
%matplotlib widget

# specs
frames = (6429, )     # amyloid, mica, dark

# creating objects first file
file1 = '2029_Dentate_gyrus-roi0_masked-previous-qgrid.h5'
qgrid1 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])
dt1  = h5xs(f'{file1}', transField="em2_sum_all_mean_value")
sn = h5_top_group(file1)
dt1.load_d1s(sn=sn)

# creating objects 2nd file
file2 = '2029_Dentate_gyrus-roi0_masked-exp-qgrid.h5'
qgrid2 = np.array([0.005, 0.006, 0.007, 0.008, 0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026, 0.027, 0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035, 0.036, 0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044, 0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.052, 0.054, 0.056, 0.058, 0.06 , 0.062, 0.064, 0.066, 0.068, 0.07 , 0.072, 0.074, 0.076, 0.078, 0.08 , 0.082, 0.084, 0.086, 0.088, 0.09 , 0.092, 0.094, 0.096, 0.098, 0.1  , 0.105, 0.11 , 0.115, 0.12 , 0.125, 0.13 , 0.135, 0.14 , 0.145, 0.15 , 0.155, 0.16 , 0.165, 0.17 , 0.175, 0.18 , 0.185, 0.19 , 0.195, 0.2  , 0.205, 0.21 , 0.215, 0.22 , 0.225, 0.23 , 0.235, 0.24 , 0.245, 0.25 , 0.255, 0.26 , 0.265, 0.27 , 0.275, 0.28 , 0.285, 0.29 , 0.295, 0.3  , 0.305, 0.31 , 0.315, 0.32 , 0.325, 0.33 , 0.335, 0.34 , 0.345, 0.35 , 0.355, 0.36 , 0.365, 0.37 , 0.375, 0.38 , 0.385, 0.39 , 0.395, 0.4  , 0.405, 0.41 , 0.415, 0.42 , 0.425, 0.43 , 0.435, 0.44 , 0.445, 0.45 , 0.455, 0.46 , 0.465, 0.47 , 0.475, 0.48 , 0.485, 0.49 , 0.495, 0.5  , 0.51 , 0.52 , 0.53 , 0.54 , 0.55 , 0.56 , 0.57 , 0.58 , 0.59 , 0.6  , 0.61 , 0.62 , 0.63 , 0.64 , 0.65 , 0.66 , 0.67 , 0.68 , 0.69 , 0.7  , 0.71 , 0.72 , 0.73 , 0.74 , 0.75 , 0.76 , 0.77 , 0.78 , 0.79 , 0.8  , 0.81 , 0.82 , 0.83 , 0.84 , 0.85 , 0.86 , 0.87 , 0.88 , 0.89 , 0.9  , 0.91 , 0.92 , 0.93 , 0.94 , 0.95 , 0.96 , 0.97 , 0.98 , 0.99 , 1.   , 1.03 , 1.06 , 1.09 , 1.12 , 1.15 , 1.18 , 1.21 , 1.24 , 1.27 , 1.3  , 1.33 , 1.36 , 1.39 , 1.42 , 1.45 , 1.48 , 1.51 , 1.54 , 1.57 , 1.6  , 1.63 , 1.66 , 1.69 , 1.72 , 1.75 , 1.78 , 1.81 , 1.84 , 1.87 , 1.9  , 1.93 , 1.96 , 1.99 , 2.02 , 2.05 , 2.08 , 2.11 , 2.14 , 2.17 , 2.2  , 2.23 , 2.26 , 2.29 , 2.32 , 2.35 , 2.38 , 2.41 , 2.44 , 2.47 , 2.5  , 2.53 , 2.56 , 2.59 , 2.62 , 2.65 , 2.68 , 2.71 , 2.74 , 2.77 , 2.8  , 2.83 , 2.86 , 2.89 , 2.92 , 2.95 , 2.98 , 3.01 , 3.04 , 3.07 , 3.1  , 3.13 , 3.16 , 3.19 ])
dt2  = h5xs(f'{file2}', transField="em2_sum_all_mean_value")
sn = h5_top_group(file2)
dt2.load_d1s(sn=sn)

# creating objects 3rd file
file3 = '2029_Dentate_gyrus-roi0_masked.h5'
dt3  = h5xs(f'{file3}', transField="em2_sum_all_mean_value")
qgrid3 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])
sn = h5_top_group(file3)
dt3.load_d1s(sn=sn)

scattering = '_WAXS2'
xscale = 'linear'
yscale = 'log'

f, axs = plt.subplots(3, 1, figsize=(12,8), num=f'{file}')
axs[0].plot(qgrid1, dt1.d1s[sn][scattering][frames[0]].data, label = f'{file1}frame-{frames[0]}')
axs[0].plot(qgrid3, dt3.d1s[sn][scattering][frames[0]].data, label = f'{file3}frame-{frames[0]}')
axs[0].set(title = f'{scattering} {file1}', xscale= xscale, yscale= yscale)
axs[0].legend()
axs[1].plot(qgrid2, dt2.d1s[sn][scattering][frames[0]].data, label = f'frame-{frames[0]}')
axs[1].set(title = f'{scattering} {file2}', xscale= xscale, yscale= yscale)
axs[2].plot(qgrid3, dt3.d1s[sn][scattering][frames[0]].data, label = f'frame-{frames[0]}')
axs[2].set(title = f'{scattering} {file3}', xscale= xscale, yscale= yscale)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
# previous q grid
idx_l, idx_u, valid_diff_values = valid_idx_search(qgrid1, dt1.d1s[sn]['_WAXS2'][frames[0]].data.reshape(1,-1),show_q=True)
print(idx_l, idx_u)
print('total WAXS points - ', len(qgrid1[idx_l:idx_u]))

# present q grid
idx_l, idx_u, valid_diff_values = valid_idx_search(qgrid2, dt2.d1s[sn]['_WAXS2'][frames[0]].data.reshape(1,-1),show_q=True)
print(idx_l, idx_u)
print('total WAXS points - ', len(qgrid1[idx_l:idx_u]))

low valid idx = 87, low valid Q = 0.185, high valid idx = 617 , high valid Q = 2.835
87 618
total WAXS points -  531
low valid idx = 87, low valid Q = 0.185, high valid idx = 271 , high valid Q = 3.130
87 272
total WAXS points -  185


In [3]:
### background subtraction
%matplotlib widget
from essential_func import *
from py4xs.hdf import h5xs
from py4xs.slnxs import trans_mode

print(f'{os.getcwd()}')

# specs
# file = '1934_SN_masked.h5'  
# frames = (4518, 3687, 3045)     # amyloid, mica, dark  ---> frames can be choosen from SAXS heat map

file = '2029_Dentate_gyrus-roi0-repack.h5'
frames = (2080, 1358, 2223, )     # amyloid, mica, dark
show_eb = False                  # show errorbar in the plot

## semi-spec
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

# creating objects
dt  = h5xs(f'{file}', transField="em2_sum_all_mean_value")
sn = h5_top_group(file)
dt.load_d1s(sn=sn)

## heat map from file
scatterings = ('_SAXS', '_WAXS2')
f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings)

### 1-d plot for bright, moderate and dark frames
scatterings = ('_SAXS',          '_WAXS2',        'merged',    )
colors =      (('r','b','g'), ('r','b','g'), ('r+','bo','g--'),)
f, axs = plt.subplots(1, 3, figsize=(15,5), num=f'{file}')
xscale = 'linear'
yscale = 'log'

for i, (scattering, color) in enumerate(zip(scatterings,colors)):
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[0]].data, color[0], label = f'bright {frames[0]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[1]].data, color[1], label = f'moderate {frames[1]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[2]].data, color[2], label = f'dark {frames[2]}')
    axs[i].set(title = f'{scattering} {file}', xscale= xscale, yscale= yscale)
    axs[i].legend()

## background correction calculations
sc_factor=1.

dset = dt.d1s[sn][scattering][frames[0]]
dbak = dt.d1s[sn][scattering][frames[2]]

dt.set_trans(transMode=trans_mode.external)
sc  = dset.trans / dbak.trans
print(f'Scaling factor, {dset.trans}/{dbak.trans} = {sc}')
Iq  = dset.data - dbak.data * sc * sc_factor
dI  = dset.err  + dbak.err * sc * sc_factor
idx = (dset.data > 0) & (dbak.data > 0)

## plotting settings
f, axs = plt.subplots(1,2,figsize=(16,4))
plt.subplots_adjust(bottom=0.15)

axs[0].plot(dbak.qgrid[idx], dset.data[idx], color[0], label = f'bright {frames[0]}')
axs[0].plot(dbak.qgrid[idx], dbak.data[idx], color[2], label = f'dark-bcg {frames[2]}')

if show_eb:
    axs[0].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx])
    axs[1].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx], label = f'{frames[0]} bcor')
else:
    axs[0].plot(dbak.qgrid[idx], Iq[idx], label = 'bcor')
    axs[1].plot(dbak.qgrid[idx], Iq[idx], label = f'{frames[0]} bcor')

axs[0].set(xlabel = "$q (\AA^{-1})$", ylabel = "$I$", title = f'Background correction {file} errorbar = {show_eb}', xscale = xscale, yscale = yscale, )
axs[1].set(xscale = xscale, yscale = yscale, title = f'Final Background Subtracted {file} errorbar = {show_eb}' , )
axs[0].legend()
axs[1].legend()
print(Iq[1:10])

/Users/bashit.a/Documents/Alzheimer/Nov-2021


KeyError: "Unable to open object (object '2029_Dentate_gyrus-roi0-repack' doesn't exist)"

In [3]:
### learn this part of the code
def flatten(x):
    """
    l = [2,[[1,2]],1]
    list(flatten(l))
    """
    for item in x:
        if type(item) in [list]:
            for num in flatten(item):
                yield(num)
        else:
            yield(item)

flatten=lambda l: sum(map(flatten,l),[]) if isinstance(l,list) else [l]
l = [2,[[1,2]],1]
flatten(l)

[2, 1, 2, 1]

In [1]:
### Finding Radius of gyration
from py4xs.hdf import h5xs,h5exp,lsh5,proc_d1merge
from py4xs.data2d import Data2d
from py4xs.slnxs import trans_mode
import numpy as np
import pylab as plt
import warnings,json

warnings.filterwarnings('ignore')

dt0  = h5xs("mica.h5", transField="em2_sum_all_mean_value")
dt  = h5xs("2048_B8.h5", transField="em2_sum_all_mean_value")

dt.load_d1s()
dt0.load_d1s()

dt.set_trans(transMode=trans_mode.external)
dt0.set_trans(transMode=trans_mode.external)

print(dt.d1s['2048_B8']['merged'][2080].trans)
print(dt0.d1s['mica']['merged'][10].trans)
#print(dt.d1s['2048_B8']['merged'][2080].overlaps)
#dt0.d1s['mica']['merged'][10].plot()

OSError: Unable to open file (unable to open file: name = 'mica.h5', errno = 2, error message = 'No such file or directory', flags = 1, o_flags = 2)

In [20]:
td = np.vstack((dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data))
qs = 0.007
qe = 10
rg = 15
# td = td[:, td[0, :] >= qs]     # find td for  greater than q=0.007
# td = td[:, td[0, :] <= qe]     # find td for smaller than q=1.3/1.5=0.086
# td[0, :] = td[0, :] * td[0, :]
# td[1, :] = np.log(td[1, :])
# rg, i0 = np.polyfit(td[0, :], td[1, :], 1)

In [21]:
from numpy.polynomial import polynomial as P
cnt = 0
while cnt < 10:
    if qe > 1.3/rg and 1.3/rg > qs+0.004: 
        qe = 1.3/rg
    td = np.vstack((dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data))
    td = td[:, td[0, :] >= qs]     # find td for  greater than q=0.007
    td = td[:, td[0, :] <= qe]     # find td for smaller than q=1.3/1.5=0.086
    td[0, :] = td[0, :] * td[0, :]
    td[1, :] = np.log(td[1, :])
    rg, i0 = np.polyfit(td[0, :], td[1, :], 1)
    i0 = np.exp(i0)
    
    print(cnt, qe, rg, i0)
    
    if rg<0:
        rg = np.sqrt(-rg * 3.)
    else:
        rg = 1e-6   # 
        print("likely strong inter-particle interaction ...")
        break
    cnt += 1
print(rg)

0 0.08666666666666667 -831.264016704801 20.377926211495332
1 0.026032341584365102 -6234.912831830037 281.9576616244667
2 0.026032341584365102 -6234.912831830037 281.9576616244667
3 0.026032341584365102 -6234.912831830037 281.9576616244667
4 0.026032341584365102 -6234.912831830037 281.9576616244667
5 0.026032341584365102 -6234.912831830037 281.9576616244667
6 0.026032341584365102 -6234.912831830037 281.9576616244667
7 0.026032341584365102 -6234.912831830037 281.9576616244667
8 0.026032341584365102 -6234.912831830037 281.9576616244667
9 0.026032341584365102 -6234.912831830037 281.9576616244667
136.76526786977064


In [72]:
%matplotlib widget
qmax = 5;
dmax=200.;

if dt.qgrid[-1] < qmax:
    qmax = dt.qgrid[-1]

tqgrid = np.arange(0, qmax, qmax / len(dt.qgrid))
tint = np.interp(tqgrid, dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data)
print(tint.shape)
tint[tqgrid * rg < 1.] = i0 * np.exp(-(tqgrid[tqgrid * rg < 1.] * rg) ** 2 / 3.)
#print(tint)

# tint -= tint[-10:].sum()/10
# Hanning window for reducing fringes in p(r)
tw = np.hanning(2 * len(tqgrid) + 1)[len(tqgrid):-1]

# plotting hanning window
tmp = np.hanning(2 * len(tqgrid) + 1)
#plt.plot(np.arange(0,len(tint[tqgrid * rg < 1.])), tint[tqgrid * rg < 1.])
#plt.yscale('log')
tint *= tw
#plt.show()

trgrid = np.arange(0, dmax, 1.)
kern = np.asmatrix([[rj ** 2 * np.sinc(qi * rj / np.pi) for rj in trgrid] for qi in tqgrid])
tt = np.asmatrix(tint * tqgrid ** 2).T
tpr = np.reshape(np.array((kern.T * tt).T), len(trgrid))
tpr /= tpr.sum()
print(tpr.shape)

(690,)
(200,)


In [39]:
tqgrid = np.arange(0, qmax, qmax / len(dt.qgrid))
tint = np.interp(tqgrid, dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data)
tqgrid[0:10]

In [11]:
%matplotlib widget
file = '2302_EC-roi0.h5'
with h5py.File(file,'r') as hdf:
    xps = hdf.get(file.split('.h5')[0]  + '/xsp3_spectrum_array_data_monitor/data')
    xps = np.array(xps.get('xsp3_spectrum_array_data'))
print(xps.shape, xps.min(), xps.max())
plt.imshow(xps.reshape(-1, 4096))

(15495168,) 0.0 552.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [198]:
%matplotlib widget
### image registration
import cv2
from pystackreg import StackReg
import matplotlib.pyplot as plt


#load reference and "moved" image
ref = cv2.imread('unstained-2048_B8.png', cv2.IMREAD_GRAYSCALE)
Height, Width = ref.shape
mov = cv2.imread('tau-stained-2048_B8.png', cv2.IMREAD_GRAYSCALE)
mov = cv2.resize(mov, (Width,Height))

#Rigid Body transformation
sr = StackReg(StackReg.RIGID_BODY)
out_rot = sr.register_transform(ref, mov)
plt.imshow(out_rot, cmap='gray')

AttributeError: 'NoneType' object has no attribute 'shape'

In [206]:
%matplotlib widget

import glob
import tifffile
import numpy
import cv2
from pystackreg import StackReg
import matplotlib.pyplot as plt
os.chdir('/Users/bashit.a/Documents/Alzheimer/Codes')

with tifffile.TiffWriter('stack_images/my_image_stack.tif') as stack:
    for filename in glob.glob('stack_images/*.tif'): 
        stack.save(tifffile.imread(filename))

#load reference and "moved" image
ref = cv2.imread('stack_images/my_image_stack.tif')

#Rigid Body transformation
sr = StackReg(StackReg.RIGID_BODY)
out_previous = sr.register_transform_stack(ref, reference="first")
out_previous_int = out_previous.astype(numpy.int8)
                                           
tifffile.imsave('stack_images/my_aligned_stack.tif', out_previous_int)
plt.imshow(out_previous_int, cmap='gray')

<ipython-input-206-f25803115897>:13: DeprecationWarning: TiffWriter: data with shape (756, 1033, 3) and dtype 'int8'' are stored as RGB with contiguous samples. Future versions will store such data as MINISBLACK in separate pages by default unless the 'photometric' parameter is specified.
  stack.save(tifffile.imread(filename))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
%matplotlib widget
import pandas as pd
import pylab as plt
folder = '/Users/bashit.a/Documents/Alzheimer/PDB/CSV/'
file = '6hrf.csv'
df = pd.read_csv(folder+file);
plt.plot(df['q'], np.log(df['0']),label='0');
plt.plot(df['q'], np.log(df['1']),label='1');
plt.plot(df['q'], np.log(df['2']),label='2');
plt.plot(df['q'], np.log(df['3']),label='3');
plt.plot(df['q'], np.log(df['-1']),label='-1');
plt.plot(df['q'], np.log(df['-2']),label='-2');
plt.plot(df['q'], np.log(df['-3']),label='-3');
plt.title(file.split('.')[0])
plt.xlabel('q')
plt.ylabel('log(I)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
import hdf5plugin

In [6]:
%matplotlib widget
import numpy as np
from mpl_toolkits import mplot3d
import warnings,json

plt.style.use('seaborn-poster')
cmap = plt.cm.jet

os.chdir('/Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sorted/')   # July-2021/BNL-Data/sample-3/  July-2021/BNL-Data/sorted/
file = "1892_EC-roi0_masked.h5"                                              # '1892_FC-roi0_masked.h5'      # "2029_Dentate_gyrus-roi0_masked.h5"
dt  = h5xs(file, transField="em2_sum_all_mean_value")

fig  = plt.figure(figsize=(15,5))
axs0 = fig.add_subplot(1, 3, 1)
axs1 = fig.add_subplot(1, 3, 2)
axs2 = fig.add_subplot(1, 3, 3, projection='3d')

x = dt.fh5[f"{dt.samples[0]}/primary/data/ss_sx"][...]
y = dt.fh5[f"{dt.samples[0]}/primary/data/ss_sz"][...]
print('average difference of a y-axis = ' , np.abs(np.sum( y[1:] - y[:-1])/(len(y)-1)))

Iq = read_Iq(file, scattering='_WAXS2')
z = find_rep_value(qgrid2, Iq , args=1.24, method = 'point')  # np.ones(len(x))  find_rep_value(qgrid2, Iq , args=1.24, method = 'point')
# z = np.clip(z,0, 0.07)
axs0.scatter(x, y, c =z, cmap = cmap, marker ='.', s=60)

# Set axes label
axs0.set_xlabel('x', fontsize=9)
axs0.set_ylabel('y', fontsize=9)
axs0.tick_params(axis='x', labelsize=8)
axs0.tick_params(axis='y', labelsize=8)

dt  = h5xs(file, transField="em2_sum_all_mean_value")

header = json.loads(dt.fh5[dt.samples[0]].attrs['start'])
print(header['snaking'])

xpos = dt.fh5[f"{dt.samples[0]}/primary/data/ss_sx"][...]
ypos = dt.fh5[f"{dt.samples[0]}/primary/data/ss_sz"][...]

axs1.plot(xpos, ypos, linewidth=0.5)   
# axs1.scatter(xpos, ypos, s = 3)
axs1.tick_params(axis='x', labelsize=8)
axs1.tick_params(axis='y', labelsize=8)

axs2.scatter3D(x, y, z, c =z, cmap = cmap, marker ='o', s=40)
axs2.view_init(azim=-90, elev=90)
axs2.set_xlabel('x', labelpad=20, fontsize=9)
axs2.set_ylabel('y', labelpad=20, fontsize=9)
axs2.set_zlabel('z', labelpad=20, fontsize=9)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)


plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

average difference of a y-axis =  1.5362949683938831e-09
qvalue is : 1.2400
Minimum, Maximum Iq : 0.0214, 0.0769
[False, True]


In [8]:
%matplotlib widget

file = "2029_Dentate_gyrus-roi0_masked.h5"
dt  = h5xs(file, transField="em2_sum_all_mean_value")

header = json.loads(dt.fh5[dt.samples[0]].attrs['start'])
Height, Width = header['shape']
npatterns = Height*Width
print(header['snaking'])
print(f'Width = {Width}, Height = {Height}, Total diff. patterns = {npatterns}')

xpos = dt.fh5[f"{dt.samples[0]}/primary/data/ss_sx"][...]
ypos = dt.fh5[f"{dt.samples[0]}/primary/data/ss_sz"][...]

print(int(len(y)/Height))
fr = np.full(npatterns,0)
inc = np.arange(0, npatterns, Width)
print(inc)
for i,_ in enumerate(inc[:-1]):
    fr[inc[i]:inc[i+1]] = np.full(Width,np.random.randint(low=30,high=150))

frs = np.arange(0,npatterns,1)
xmin = np.min(x); xmax = np.max(x); 
ymin = np.min(y); ymax = np.max(y); 


def format_coord(x,y):
    if xmin <= x <= xmax and ymin <= y <= ymax:
        A = frs[np.isclose(y,  ypos, atol=1e-03,)]
        B = frs[np.isclose(x,  xpos, atol=1e-03,)]
        return 'x=%1.2f, y=%1.2f, FRAME=%d' % (x,y,np.array(list(set(A) & set(B)), dtype=int))
    else:
        return 'x=%1.2f, y=%1.2f, FRAME=UNDETECTED' % (x,y)

fig, ax = plt.subplots(1,1,figsize=(10,5))
ax.scatter(xpos, ypos, c=fr, marker = '.', s=100)
# ax.plot(xpos, ypos,)
ax.format_coord = format_coord
plt.tight_layout()

[False, True]
Width = 63, Height = 124, Total diff. patterns = 7812
63
[   0   63  126  189  252  315  378  441  504  567  630  693  756  819
  882  945 1008 1071 1134 1197 1260 1323 1386 1449 1512 1575 1638 1701
 1764 1827 1890 1953 2016 2079 2142 2205 2268 2331 2394 2457 2520 2583
 2646 2709 2772 2835 2898 2961 3024 3087 3150 3213 3276 3339 3402 3465
 3528 3591 3654 3717 3780 3843 3906 3969 4032 4095 4158 4221 4284 4347
 4410 4473 4536 4599 4662 4725 4788 4851 4914 4977 5040 5103 5166 5229
 5292 5355 5418 5481 5544 5607 5670 5733 5796 5859 5922 5985 6048 6111
 6174 6237 6300 6363 6426 6489 6552 6615 6678 6741 6804 6867 6930 6993
 7056 7119 7182 7245 7308 7371 7434 7497 7560 7623 7686 7749]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
### WAXS frame removal --------
file = '2428_FC-roi1.h5'          # must input source file name
frame = 3740                      # frame to be replaced
value = -1                        # value to be replaced with

def waxs_frame_edit(file, frame, value):
   
    ### read/write hdf file
    with h5py.File(file,'r+') as hdf:
        dset = hdf.get(f'{h5_top_group(file)}/primary/data')
        dset['pilW2_image'][frame] = np.full_like(dset['pilW2_image'][frame],value)
    print(f'{file} WAXS {frame} is replaced by {value}')

waxs_frame_edit(file, frame, value)

2428_FC-roi1.h5 WAXS 3740 is replaced by -1
